# file_db.py

In [1]:
# 引入标准库和第三方库
# 用于跨平台地处理文件路径（如查找文件、拼接路径）
from pathlib import Path
# 用于加载句子嵌入模型并生成句子向量
from sentence_transformers import SentenceTransformer
# Chroma 是一个轻量级的向量数据库，支持相似度检索等操作
import chromadb  
# 用于获取当前时间，常用于记录创建时间等元信息
from datetime import datetime

# 加载本地已经下载好的句子嵌入模型
# 模型路径是手动从 ModelScope 下载并保存到本地的中文 BGE 模型
# 注意路径中有 "___" 替代了 "."，这是 ModelScope 的默认命名方式
model = SentenceTransformer(r'model_dir\BAAI\bge-large-zh-v1___5')  

# 创建一个 Chroma 的数据库客户端，用于向量存储和检索
# 使用 PersistentClient 表示采用“文件持久化模式”，数据将存储在指定目录
# 如果使用 chromadb.Client()，那就是内存模式，程序结束后数据就消失了
# 数据库存储在当前目录下的 chroma_v2 文件夹中
client = chromadb.PersistentClient('./chroma_v2')

# 从数据库中获取一个名为 "da_fei" 的集合（collection），如果不存在就自动创建
# 每个 collection 就像一个“表”，用来存储一组相似用途的向量数据
collecton = client.get_or_create_collection(
    name = 'da_fei',  # 集合的名称，唯一标识
    metadata={   # 集合的元数据，用于描述集合的信息
        "介绍": "文本文件的向量数据库",   # 自定义描述，说明这个集合用于保存哪些数据
        "创建时间": str(datetime.now()),  # 获取当前时间作为创建时间，转为字符串格式
        "hnsw:space": "cosine" # 设置检索时采用的距离度量方式为 cosine（余弦距离）
        # cosine 距离 = 1 - 余弦相似度，值越小，表示越相似
    }
)

D:\11_Anaconda\envs\RAG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


# simple_rag_v2.py

In [2]:
# 从 openai 库中导入 OpenAI，用于调用 deepseek-chat 模型
from openai import OpenAI
# 导入 os 模块，用于读取系统环境变量
import os
# 从 pathlib 模块导入 Path，用于进行文件搜索操作
from  pathlib import Path

# 从环境变量中获取名为 DEEPSEEK_API_KEY 的密钥（你需提前设置这个环境变量）
api_key = os.environ['DEEPSEEK_API_KEY'] 

# 创建一个 OpenAI 客户端对象，设置 base_url 为 DeepSeek 的 API 地址，同时传入 API 密钥
client = OpenAI(base_url = 'https://api.deepseek.com', api_key=api_key) 

# 步骤1：语义检索函数，根据用户输入的 query 在向量数据库中检索相似内容
def retrieval(query):
    # 初始化上下文内容为空
    context = ''
    # 将单个查询字符串变成列表，以便编码器处理
    query = [query]

    # 使用 embedding 模型将查询转换为向量表示
    query_embedding = model.encode(query)

    # 在向量数据库中进行查询，查找最相似的5条内容
    data = collecton.query(query_embedding.tolist(), n_results=5)
    # print(data)

    # 在 ChromaDB 中，使用余弦相似度计算距离，返回的 distances 实际为 (1 - 相似度)
    # 获取返回的文本列表（第一条 query 的结果）
    text_list = data['documents'][0]

    # 遍历所有文本块
    for t in text_list:
        # 拼接每段文本到 context 中
        context += t
        # 添加分隔符，便于后续模型理解多段内容
        context += '\n----------\n'

    # 返回拼接好的上下文内容
    return context


# 步骤 2：根据检索到的 context 增强查询 query，构建提示 prompt
def augmented(query, context=''):
    if not context:
        # 如果上下文为空，只返回简短的问题提示
        return f'请简要回答下面问题: {query}'
    else:
        # 如果有上下文，构建带上下文的提示，指导模型根据上下文来回答
        prompt = f'''请根据上下文信息来回答问题，如果上下文信息不足以回答问题，请直接说:‘根据上下文信息，无法回答问题’
    上下文:
    {context}
    
    问题:{query}'''

        #检查长度是否超过上下文上限
        return prompt

# 步骤 3：调用 deepseek-chat 模型生成回答
def generation(prompt):
    # 向模型发送聊天请求
    completion = client.chat.completions.create(    
        # 使用 DeepSeek 提供的聊天模型
        model='deepseek-chat',  
        # 消息列表，仅含一条用户提问
        messages=[  
            {
                'role': 'user',
                'content': prompt  # 用户提问内容
            }
        ],
    )
    # 返回模型回复的文本内容
    return completion.choices[0].message.content#输出结果


if __name__ == '__main__':
    # 示例：提问与知识库有关（命中）
    query = '能飞多久？' 
    
    print('====== 不使用RAG=======')
    # 不带上下文直接问问题
    print(generation(query))


    print('======= 使用RAG========')
    # 使用 RAG：先从知识库检索上下文，再构建提示，再生成回答
    # 步骤 1：检索上下文
    context = retrieval(query)
    # 步骤 2：构建提示词
    prompt = augmented(query, context)
    # 步骤 3：生成回答
    resp = generation(prompt)
    # 输出结果
    print(resp)

====== 不使用RAG=======


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


“能飞多久？”这个问题的答案取决于具体的飞行对象或设备。以下是不同情境下的常见解释：

---

### 1. **无人机/航拍器**
   - **消费级无人机**（如大疆Mavic系列）：通常续航时间为 **20-30分钟**（受电池容量、风速、载重影响）。
   - **专业级无人机**：部分型号可达 **40-60分钟**（如大疆Matrice 300）。
   - **提示**：实际飞行需预留返航电量（建议剩余20%电量时返航）。

---

### 2. **商用客机**
   - **短途航班**（如波音737）：约 **3-5小时**（航程约3,000-5,000公里）。
   - **长途航班**（如空客A350）：可达 **16-20小时**（航程约15,000-18,000公里）。
   - **影响因素**：载油量、载客量、飞行高度、天气等。

---

### 3. **战斗机**
   - **无空中加油**：通常 **1-3小时**（如F-16约2小时，开启加力燃烧耗油更快）。
   - **有空中加油**：可大幅延长（如远程任务可达10小时以上）。

---

### 4. **鸟类**
   - **小型鸟**（如麻雀）：连续飞行约 **1-2小时**。
   - **迁徙鸟类**（如斑尾塍鹬）：可不停歇飞行 **7-10天**（借助气流，跨洋迁徙）。

---

### 5. **其他飞行器**
   - **热气球**：约 **1-3小时**（受燃料和天气限制）。
   - **太阳能无人机**（如NASA“太阳神”）：理论可达 **数月**（依靠太阳能持续供电）。

---

### 注意事项
- **电池/燃料**：电动设备受电池容量限制，燃油设备需考虑油耗率。
- **环境条件**：逆风、温度、海拔均会影响续航。
- **法规限制**：无人机需遵守当地空域法规（如最高飞行时间、高度等）。

如果需要更具体的答案，请提供更多背景信息（如设备型号、飞行目的等）！
======= 使用RAG========
探索者X100型无人机的电池续航时间长达30分钟。
